In [1]:
! pip install textattack  sentence_transformers evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 418 kB 31.5 MB/s 
     |████████████████████████████████| 85 kB 6.0 MB/s 
     |████████████████████████████████| 72 kB 1.6 MB/s 
     |████████████████████████████████| 125 kB 93.0 MB/s 
     |████████████████████████████████| 769 kB 85.6 MB/s 
     |████████████████████████████████| 401 kB 95.0 MB/s 
     |████████████████████████████████| 3.6 MB 76.9 MB/s 
     |████████████████████████████████| 5.8 MB 76.6 MB/s 
     |████████████████████████████████| 41.4 MB 1.1 MB/s 
     |████████████████████████████████| 365 kB 90.7 MB/s 
     |████████████████████████████████| 60 kB 9.8 MB/s 
     |████████████████████████████████| 95 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 98.8 MB/s 
     |████████████████████████████████| 132 kB 98.7 MB/s 
     |████████████████████████████████| 182 kB 98.2 MB/s 
     |████████████████████████████████|

In [2]:
import textattack
import pandas as pd
import json

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from datasets import load_dataset, load_from_disk, concatenate_datasets

In [4]:
from textattack.goal_functions import UntargetedClassification
from textattack.datasets import HuggingFaceDataset
from tqdm import tqdm # tqdm provides us a nice progress bar.
from textattack.loggers import CSVLogger # tracks a dataframe for us.
from textattack.attack_results import SuccessfulAttackResult
from textattack import Attacker
from textattack import AttackArgs
from textattack.datasets import Dataset

# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from datasets import load_dataset

In [7]:
root = "/content/drive/Shareddrives/BipinA/EE8228neural_info/Project"


In [8]:
from datasets import load_from_disk

In [10]:
dataset = load_from_disk(f"{root}/data/imdb_test_6k.hf")

In [12]:
_dataset = dataset.shuffle(seed=42).select(range(1000))

In [16]:
test_dataset = HuggingFaceDataset(_dataset.select(range(500,1000)))

In [17]:
len(test_dataset)

500

In [18]:
checkpoint_name = "textattack/bert-base-uncased-imdb"

model = transformers.AutoModelForSequenceClassification.from_pretrained(checkpoint_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint_name)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

# Create the goal function using the model
goal_function = UntargetedClassification(model_wrapper)


Downloading:   0%|          | 0.00/511 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [19]:
import torch

# Attack 1: BAEGarg2019

In [20]:
torch.cuda.empty_cache()

In [21]:
from textattack.attack_recipes import BAEGarg2019

In [22]:
attack = BAEGarg2019.build(model_wrapper)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [23]:
import numpy as np

In [24]:
import torch

In [25]:
import os 

In [26]:
torch.cuda.device_count()

1

In [27]:
root

'/content/drive/Shareddrives/BipinA/EE8228neural_info/Project'

In [28]:
attack_args = AttackArgs(num_examples=-1,
                        parallel = bool(torch.cuda.device_count()),
                        log_to_csv=f"{root}/data/log_BAEGarg2019_attack_BERT_2.csv",
                        checkpoint_interval=100, 
                        checkpoint_dir=f"{root}/checkpoints_log_BAEGarg2019_attack_BERT_2",
                        csv_coloring_style = 'plain',
                        disable_stdout=True,
                        shuffle=True,
                        enable_advance_metrics=True,
                        silent=True
                        )

attacker = Attacker(attack, test_dataset, attack_args)

attack_results = attacker.attack_dataset()

[Succeeded / Failed / Skipped / Total] 57 / 37 / 6 / 100:  20%|██        | 100/500 [26:33<1:46:12, 15.93s/it]

[Succeeded / Failed / Skipped / Total] 128 / 60 / 12 / 200:  40%|████      | 200/500 [50:32<1:15:48, 15.16s/it]

[Succeeded / Failed / Skipped / Total] 181 / 101 / 18 / 300:  60%|██████    | 300/500 [1:12:40<48:26, 14.53s/it]

[Succeeded / Failed / Skipped / Total] 244 / 134 / 22 / 400:  80%|████████  | 400/500 [1:32:16<23:04, 13.84s/it]

[Succeeded / Failed / Skipped / Total] 298 / 169 / 33 / 500: 100%|██████████| 500/500 [1:58:12<00:00, 14.19s/it]

...

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (99237 > 1024). Running this sequence through the model will result in indexing errors


In [30]:
dir(attack_results)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']